#### 參考 Hands-On Machine Learning with Scikit-Learn and TensorFlow. page 235 - 242
* Linear Regression using Tensorflow

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

/Users/eric/.pyenv/versions/3.5.2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/eric/.pyenv/versions/3.5.2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [12]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")  # reshape: only one column, but unknown row (until the end)

In [14]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # matmul: np.dot


In [16]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [19]:
def meanNormal(X):
    X = (X - np.mean(X)) / (np.max(X) - np.min(X))
    return X

scaled_housing_data_plus_bias = meanNormal(housing_data_plus_bias)

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [37]:
n_epochs = 1000
learning_rate = 0.01

In [38]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #shape, min, max
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)

In [39]:
# assign update value from Gradient Descent
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [40]:
init = tf.global_variables_initializer()

In [41]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch", epoch, "MSE =", mse.eval())
            sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 6.971133
Epoch 100 MSE = 6.896692
Epoch 200 MSE = 6.825523
Epoch 300 MSE = 6.75747
Epoch 400 MSE = 6.6923842
Epoch 500 MSE = 6.630134
Epoch 600 MSE = 6.57057
Epoch 700 MSE = 6.513584
Epoch 800 MSE = 6.459046
Epoch 900 MSE = 6.4068465


In [35]:
housing_data_plus_bias

array([[   1.        ,    8.3252    ,   41.        , ...,    2.55555556,
          37.88      , -122.23      ],
       [   1.        ,    8.3014    ,   21.        , ...,    2.10984183,
          37.86      , -122.22      ],
       [   1.        ,    7.2574    ,   52.        , ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.        ,    1.7       ,   17.        , ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.        ,    1.8672    ,   18.        , ...,    2.12320917,
          39.43      , -121.32      ],
       [   1.        ,    2.3886    ,   16.        , ...,    2.61698113,
          39.37      , -121.24      ]])

In [42]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)


In [43]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch", epoch, "MSE =", mse.eval())
            sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 7.2882533
Epoch 100 MSE = 7.1822677
Epoch 200 MSE = 7.082116
Epoch 300 MSE = 6.987462
Epoch 400 MSE = 6.897947
Epoch 500 MSE = 6.813276
Epoch 600 MSE = 6.7331257
Epoch 700 MSE = 6.657247
Epoch 800 MSE = 6.585355
Epoch 900 MSE = 6.517245


In [44]:
np.ceil(m / 100)

207.0

In [46]:
20640/100

206.4